In [1]:
import numpy as np
import pandas as pd

In [2]:
filename="dataset2.txt"
df=pd.read_csv(filename,header=None,error_bad_lines=False,encoding='utf8')

/tmp/ipykernel_6521/3027809746.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv(filename,header=None,error_bad_lines=False,encoding='utf8')
b'Skipping line 7421: expected 4 fields, saw 6\n'


In [3]:
df = df[1:]
df = df.drop(columns = [2,3])
message = df.drop(columns = [0])

In [4]:
message[['time', 'chat']] = message[1].str.split('-', n=1, expand=True)
message = message.drop(columns = [1])
df.columns=['Date', 'Chat']
df['Time']=message['time']
df['content']=message['chat']
df = df.drop(columns = ['Chat'])
df['content']=df['content'].str.lower()
df

,Date,Time,content
1,1/20/22,10:23,himanshu ai: hlo bro
2,1/20/22,10:23,himanshu ai: i am himanshu from ai 2021😅😅
3,1/20/22,10:23,himanshu ai: can you please explain me the la...
4,1/20/22,11:04,karthik: hlo
5,1/20/22,11:40,karthik: bro did u get it?
...,...,...,...
7579,5/10/23,08:12,karthik: ha
7580,5/10/23,08:13,himanshu ai: i don't have much idea but try t...
7581,5/10/23,08:13,karthik: this is second time
7582,5/10/23,08:19,himanshu ai: https://forums.linuxmint.com/vie...


In [5]:
df['content'] = df['content'].str.replace('<media omitted>','')
df['content'] = df['content'].str.replace('this message was deleted','')
df['content_len'] = df['content'].str.len()
df = df[df['content_len'] > 0]
df.shape

(7431, 4)

In [6]:
df[['author', 'message']] = df['content'].str.split(':', n=1, expand=True)
df = df.drop(columns = ['content'])

/tmp/ipykernel_6521/76138651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['author', 'message']] = df['content'].str.split(':', n=1, expand=True)
/tmp/ipykernel_6521/76138651.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['author', 'message']] = df['content'].str.split(':', n=1, expand=True)


In [7]:
df['replying_to_author'] = df['author'].shift(1)
df['replying_to'] = df['message'].shift(1)
df = df.dropna()
df

,Date,Time,content_len,author,message,replying_to_author,replying_to
2,1/20/22,10:23,42.0,himanshu ai,i am himanshu from ai 2021😅😅,himanshu ai,hlo bro
3,1/20/22,10:23,134.0,himanshu ai,can you please explain me the last problem of...,himanshu ai,i am himanshu from ai 2021😅😅
4,1/20/22,11:04,13.0,karthik,hlo,himanshu ai,can you please explain me the last problem of...
5,1/20/22,11:40,27.0,karthik,bro did u get it?,karthik,hlo
6,1/20/22,11:47,20.0,himanshu ai,no bro,karthik,bro did u get it?
...,...,...,...,...,...,...,...
7578,5/10/23,08:12,41.0,himanshu ai,it's stopping at this step?,karthik,ha
7579,5/10/23,08:12,12.0,karthik,ha,himanshu ai,it's stopping at this step?
7580,5/10/23,08:13,56.0,himanshu ai,i don't have much idea but try to do again,karthik,ha
7581,5/10/23,08:13,29.0,karthik,this is second time,himanshu ai,i don't have much idea but try to do again


In [8]:
df['is_reply'] = df['replying_to_author'] != df['author']
df = df[df['is_reply'] == 1]
df.head(5)

,Date,Time,content_len,author,message,replying_to_author,replying_to,is_reply
4,1/20/22,11:04,13.0,karthik,hlo,himanshu ai,can you please explain me the last problem of...,True
6,1/20/22,11:47,20.0,himanshu ai,no bro,karthik,bro did u get it?,True
7,1/20/22,11:48,42.0,karthik,which part u did not understand?,himanshu ai,no bro,True
8,1/20/22,11:49,55.0,himanshu ai,1st part that how program are countable,karthik,which part u did not understand?,True
9,1/20/22,11:49,11.0,karthik,k,himanshu ai,1st part that how program are countable,True


In [9]:
df['author'] == ' karthik'

4        True
6       False
7        True
8       False
9        True
        ...  
7578    False
7579     True
7580    False
7581     True
7582    False
Name: author, Length: 3992, dtype: bool

In [10]:
df_a = df[df['author'] == ' karthik'][['message','replying_to']]
df_a.columns = ['completion', 'prompt']
df_a.to_csv('data_karthik2.csv', index=False)

data_a = list(df_a.to_dict(orient='index').values())
data_a

[{'completion': ' hlo',
  'prompt': ' can you please explain me the last problem of previous discrete mathematics class that all functions are not computable.'},
 {'completion': ' which part u did not understand?', 'prompt': ' no bro'},
 {'completion': ' k', 'prompt': ' 1st part that how  program  are countable'},
 {'completion': ' ', 'prompt': ' or you can tell after the class also'},
 {'completion': ' this part',
  'prompt': ' bro i got everything except that how you concluded that set is finite if string is finite'},
 {'completion': ' 10^100 is it finite or infinite',
  'prompt': ' i watched it 3 times in video but not getting it😅😅'},
 {'completion': ' we generally think that infinite is something that has no end ',
  'prompt': ' finite'},
 {'completion': ' for environmental chemistry should we read those tables also?',
  'prompt': ' yaa it seems correct'},
 {'completion': ' hmm ok bro thank you',
  'prompt': " but krishna sir said that he can't say anything"},
 {'completion': ' tel

In [11]:
df_b = df[df['author'] == ' himanshu ai'][['message','replying_to']]
df_b.columns = ['completion', 'prompt']
df_b.to_csv('data_himanshu2.csv', index=False)
data_b = list(df_b.to_dict(orient='index').values())
data_b

[{'completion': ' no bro', 'prompt': ' bro did u get it?'},
 {'completion': ' 1st part that how  program  are countable',
  'prompt': ' which part u did not understand?'},
 {'completion': ' no problem bro', 'prompt': ' give me 5 minutes'},
 {'completion': ' bro i got everything except that how you concluded that set is finite if string is finite',
  'prompt': " i don't what is something lex order which sir mentioned in the class"},
 {'completion': ' yes', 'prompt': ' this part'},
 {'completion': ' finite', 'prompt': ' 10^100 is it finite or infinite'},
 {'completion': ' yaa it seems correct',
  'prompt': ' i thought like this this maybe wrong'},
 {'completion': ' not all',
  'prompt': ' for environmental chemistry should we read those tables also?'},
 {'completion': ' by the way where are you from',
  'prompt': ' hmm ok bro thank you'},
 {'completion': ' up', 'prompt': ' what about you'},
 {'completion': ' 😅😅', 'prompt': ' maybe more than 15 telugu people'},
 {'completion': ' hii', 'pr

In [12]:
with open('karthik2.jsonl', 'w') as f:
    for item in data_a:
        f.write("%s\n" % item)

In [13]:
with open('data_himanshu2.jsonl', 'w') as f:
    for item in data_b:
        f.write("%s\n" % item)